In [1]:
from scipy.optimize import curve_fit
import datetime as dt
import pandas as pd
import numpy as np
import altair as alt

data = pd.read_csv("corona_infections.csv", header=0, names=["day", "cases"])
data["day"] = [dt.datetime.strptime(d + ' 2020', "%b %d %Y") for d in data["day"]]
data

,day,cases
0,2020-02-15,16
1,2020-02-16,16
2,2020-02-17,16
3,2020-02-18,16
4,2020-02-19,16
5,2020-02-20,16
6,2020-02-21,16
7,2020-02-22,16
8,2020-02-23,16
9,2020-02-24,16


In [2]:
infections = alt.Chart(data).mark_line(point=True).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
infections

alt.Chart(...)

In [3]:
first_day_with_quarantine_effect = dt.datetime(year=2020, month=3, day=20)
cases_since_quarantine = np.array(
    data[data["day"] >= first_day_with_quarantine_effect]["cases"])
day_no_since_quarantine = np.array(
    [d.toordinal() for d in data[data["day"] >= first_day_with_quarantine_effect]["day"]])
print(day_no_since_quarantine)
print(cases_since_quarantine)

[737504 737505 737506 737507 737508 737509 737510 737511]
[19848 22364 24873 29056 32991 37323 43938 50871]


In [4]:
def corona_curve(x, b0, x0, k, s):
    return s * 1 / (1 + np.exp(-1 * k * s * (x - x0)) * (s / b0 - 1))

print("mins:", [0, day_no_since_quarantine[0], 1e-12, cases_since_quarantine[-1]])
print("maxs:", [cases_since_quarantine[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e2, 8.35e7])

params, _ = curve_fit(
    corona_curve, 
    xdata=day_no_since_quarantine, 
    ydata=cases_since_quarantine, 
    p0=[cases_since_quarantine[0], dt.datetime(year=2020, month=3, day=22).toordinal(), 8e-9, 5.6e7],
    bounds=(
        [0, day_no_since_quarantine[0], 1e-11, cases_since_quarantine[-1]], 
        [cases_since_quarantine[-1], dt.datetime(year=2021, month=6, day=1).toordinal(), 1e-8, 8.35e7])
)
print(params)

mins: [0, 737504, 1e-12, 50871]
maxs: [50871, 737942, 100.0, 83500000.0]
[2.67100154e+04 7.37506418e+05 4.89968463e-09 2.82800799e+07]


In [5]:
data = data[data["day"] >= dt.datetime(year=2020, month=3, day=1)]
projection_data = data.copy()
projection_data = projection_data.drop(["cases"], axis=1)
projection_data["curve"] = [corona_curve(d.toordinal(), *params) for d in projection_data["day"]]
for i in range(11):
    projection_data = projection_data.append(pd.DataFrame(
        [[list(projection_data["day"])[-1] + dt.timedelta(days=1), 
          corona_curve((list(projection_data["day"])[-1] + dt.timedelta(days=1)).toordinal(), *params)]],
        columns=("day", "curve")
    ), ignore_index=True)
projection_data

,day,curve
0,2020-03-01,1374.615816
1,2020-03-02,1578.903366
2,2020-03-03,1813.549019
3,2020-03-04,2083.063510
4,2020-03-05,2392.627602
5,2020-03-06,2748.191546
6,2020-03-07,3156.589289
7,2020-03-08,3625.669605
8,2020-03-09,4164.446646
9,2020-03-10,4783.272789


In [6]:
infections_chart = alt.Chart(data).mark_line(point=True, color="red").encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("cases:Q", title="Corona-Fallzahlen"))
projection_chart = alt.Chart(projection_data).mark_line(point=False).encode(
    alt.X("monthdate(day):O", title="Tag"),
    alt.Y("curve:Q", title="Fallzahlen (Projektion)"))
(projection_chart + infections_chart).properties(width=600, height=400)

alt.LayerChart(...)

In [7]:
bev_de = 83019213
infizierte_pro_tag = ["{:.0f}".format(bev_de*0.7/(365*dauer)) for dauer in [0.5, 1, 1.5]]
infizierte_pro_tag

['318430', '159215', '106143']